<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_dl_intro/4%20-%20Usando%20imagens%20do%20mundo%20real/19%20-%20C1_W4_Lab_3_compacted_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Efeito de imagens compactadas no treinamento

Neste notebook, você verá como a redução do tamanho alvo das imagens do gerador afetará a arquitetura e o desempenho do seu modelo. 

Essa é uma técnica útil caso você precise acelerar o treinamento ou economizar recursos de computação. Vamos começar!

Como antes, comece a fazer o download dos conjuntos de treinamento e validação:

In [ ]:
# Faça o download do conjunto de treinamento
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

In [ ]:
# Baixar o conjunto de validação
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip

Em seguida, descompacte-os:

In [ ]:
import zipfile

# Descompactar o conjunto de treinamento
local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')

# Descompactar o conjunto de validação
local_zip = './validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./validation-horse-or-human')

zip_ref.close()

Em seguida, defina os diretórios que contêm as imagens:

In [ ]:
import os

# Diretório com fotos de cavalos de treinamento
train_horse_dir = os.path.join('./horse-or-human/horses')

# Diretório com imagens humanas de treinamento
train_human_dir = os.path.join('./horse-or-human/humans')

# Diretório com imagens de cavalos de validação
validation_horse_dir = os.path.join('./validation-horse-or-human/horses')

# Diretório com imagens humanas de validação
validation_human_dir = os.path.join('./validation-horse-or-human/humans')

Você pode verificar se os diretórios não estão vazios e se o conjunto de trens tem mais imagens do que o conjunto de validação:

In [ ]:
train_horse_names = os.listdir(train_horse_dir)
print(f'CONJUNTO DE TREINO, CAVALOS: {train_horse_names[:10]}')

train_human_names = os.listdir(train_human_dir)
print(f'CONJUNTO DE TREINO, HUMANOS: {train_human_names[:10]}')

validation_horse_names = os.listdir(validation_horse_dir)
print(f'CONJUNTO DE VALIDAÇÃO, CAVALOS: {validation_horse_names[:10]}')

validation_human_names = os.listdir(validation_human_dir)
print(f'CONJUNTO DE VALIDAÇÃO, HUMANOS: {validation_human_names[:10]}')

In [ ]:
print(f'total training horse images: {len(os.listdir(train_horse_dir))}')
print(f'total training human images: {len(os.listdir(train_human_dir))}')
print(f'total validation horse images: {len(os.listdir(validation_horse_dir))}')
print(f'total validation human images: {len(os.listdir(validation_human_dir))}')

## Construir o modelo

O modelo seguirá a mesma arquitetura de antes, mas a principal diferença está no parâmetro `input_shape` da primeira camada `Conv2D`.

Como você compactará as imagens posteriormente no gerador, é necessário especificar o tamanho esperado da imagem aqui.

Portanto, em vez de 300x300 como nos dois laboratórios anteriores, você especifica uma matriz menor de 150x150.

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Observe que a forma de entrada é o tamanho desejado da imagem 150x150 com 3 bytes de cor
    # Essa é a primeira convolução
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # A segunda convolução
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # A terceira convolução
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # A quarta convolução (Você pode descomentar a 4ª e a 5ª camadas de convolução mais tarde para ver o efeito)
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
    # A quinta convolução
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
    # Achatar os resultados para alimentar um DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

Você pode ver a diferença em relação aos modelos anteriores ao imprimir o `model.summary()`.

Como esperado, haverá menos entradas para a camada `Dense` no final do modelo em comparação com os laboratórios anteriores.

Isso ocorre porque você usou o mesmo número de camadas de pooling máximo em seu modelo.

E como você tem uma imagem menor para começar (150 x 150), a saída após todas as camadas de pooling também será menor.

In [ ]:
model.summary()

Você usará as mesmas configurações para o treinamento:

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

### Pré-processamento de dados

Agora você instanciará os geradores de dados. Como mencionado anteriormente, você compactará a imagem especificando o parâmetro `target_size`. Veja a alteração simples abaixo:

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Todas as imagens serão redimensionadas em 1,/255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Imagens de treinamento de fluxo em lotes de 128 usando o gerador train_datagen
train_generator = train_datagen.flow_from_directory(
        './horse-or-human/',  # Esse é o diretório de origem das imagens de treinamento
        target_size=(150, 150),  # Todas as imagens serão redimensionadas para 150x150
        batch_size=128,
        # Como você usou a perda binary_crossentropy, você precisa de rótulos binários
        class_mode='binary')

# Imagens de treinamento de fluxo em lotes de 128 usando o gerador train_datagen
validation_generator = validation_datagen.flow_from_directory(
        './validation-horse-or-human/',  # Esse é o diretório de origem das imagens de treinamento
        target_size=(150, 150),  # Todas as imagens serão redimensionadas para 150x150
        batch_size=32,
        # Como você usou a perda binary_crossentropy, você precisa de rótulos binários
        class_mode='binary')

### Treinamento

Agora você está pronto para treinar e ver os resultados.

Anote suas observações sobre a rapidez com que o modelo é treinado e a precisão obtida nos conjuntos de treinamento e validação.

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

### Previsão do modelo

Como de costume, também é uma boa prática tentar executar seu modelo em algumas imagens escolhidas a dedo. Veja se você obteve um desempenho melhor, pior ou igual ao do laboratório anterior.

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = load_img(path, target_size=(150, 150))
  x = img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")
 

### Visualizando representações intermediárias

Você também pode olhar novamente para as representações intermediárias.

Você perceberá que a saída na última camada de convolução é ainda mais abstrata porque contém menos pixels do que antes.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import random
from tensorflow.keras.utils import img_to_array, load_img

# Defina um novo modelo que receberá uma imagem como entrada e produzirá
# representações intermediárias para todas as camadas do modelo anterior após
# a primeira.
successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

# Prepare uma imagem de entrada aleatória do conjunto de treinamento.
horse_img_files = [os.path.join(train_horse_dir, f) for f in train_horse_names]
human_img_files = [os.path.join(train_human_dir, f) for f in train_human_names]
img_path = random.choice(horse_img_files + human_img_files)
img = load_img(img_path, target_size=(150, 150))  # esta é uma imagem PIL
x = img_to_array(img)  # Matriz Numpy com formato (150, 150, 3)
x = x.reshape((1,) + x.shape)  # Matriz Numpy com formato (1, 150, 150, 3)

# Escalar de 1/255
x /= 255

# Execute a imagem na rede, obtendo assim todas as
# representações intermediárias para essa imagem.
successive_feature_maps = visualization_model.predict(x)

# Esses são os nomes das camadas, para que você possa tê-las como parte do gráfico
layer_names = [layer.name for layer in model.layers[1:]]

# Exibir as representações
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:

    # Faça isso apenas para as camadas conv / maxpool, não para as camadas totalmente conectadas
    n_features = feature_map.shape[-1]  # number of features in feature map

    # O mapa de recursos tem a forma (1, tamanho, tamanho, n_recursos)
    size = feature_map.shape[1]
    
    # Colocar as imagens em mosaico nessa matriz
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
    
      # Colocar cada filtro em uma grade horizontal grande
      display_grid[:, i * size : (i + 1) * size] = x
    
    # Exibir a grade
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

## Limpar

Execute a seguinte célula para encerrar o kernel e liberar recursos de memória:

In [ ]:
from google.colab import runtime
runtime.unassign()

## Resumo

Neste laboratório, você viu como a compactação de imagens afetou o modelo anterior. Essa é uma técnica que você deve ter em mente, especialmente quando ainda estiver na fase exploratória de seus próprios projetos. Você pode ver se um modelo menor se comporta tão bem quanto um modelo grande para que o treinamento seja mais rápido. Você também viu como é fácil personalizar suas imagens para esse ajuste de tamanho simplesmente alterando um parâmetro na classe `ImageDataGenerator`.